In [8]:
import json
import os
import pexpect
import time
from typing import Dict, Optional

In [80]:
HOME_DIR = os.path.expanduser('~')
DEFAULT_LAKE_PATH = f'{HOME_DIR}/.elan/bin/lake'
DEFAULT_LEAN_WORKSPACE = 'old-mathlib4/'

LEAN4_DEFAULT_HEADER = ""

In [61]:
def send_code_read_json(cmd, timeout_start=600, timeout_cat=600, timeout_finish=600, _child: Optional[pexpect.spawn] = None, kill=False):
        print("compiling")
        print(cmd["cmd"])
        try:
            return _send_code_read_json(cmd, timeout_start=timeout_start, timeout_cat=timeout_cat, timeout_finish=timeout_finish, _child=_child, kill=kill)
        except Exception as e:
            return {'system_error': str(e)}


def _send_code_read_json(cmd, timeout_start=600, timeout_cat=600, timeout_finish=600, _child: Optional[pexpect.spawn] = None, kill=False):
    """
    Note that there's actually no reason to make the timeouts super short. Timeouts aren't usually indicative
    of buggy code, they're just due to variance in the time it takes to run the code. So, we can just set them
    to be very long.
    """
    if _child is None:
        child = pexpect.spawn(
            f"{DEFAULT_LAKE_PATH} exe repl",
            cwd=DEFAULT_LEAN_WORKSPACE)
    else:
        child = _child

    cmd_json = json.dumps(cmd)
    print("cmd_json", cmd_json)
    child.send(cmd_json + "\r\n")
    # Read the input itself.
    # This should be printed instantly, so timeout is set to 1 second.
    child.expect_exact(cmd_json + "\r\n", timeout=timeout_cat)
    assert child.after.decode('utf-8') == cmd_json + "\r\n"
    # print("Sent code to Lean4 REPL.")

    # Read the output.
    # This code is critical; the repl seems to print out some
    # strange non-json stuff before the actual json output,
    # including characters that delete the previous input,
    # such that it doesn't show up in debug output.
    child.expect_exact("{", timeout=timeout_start)
    res = "{"
    # print("Received start of output from Lean4 REPL.")
    # while res is not a valid json string, read lines.
    # All of the lines should print essentially instantly,
    # so there are no timeouts in this loop.
    start_time = time.time()
    while True:
        res = res + child.readline().decode('utf-8')
        try:
            # print all chars in res
            json.loads(res.strip())
            break
        except json.JSONDecodeError as e:
            # print(e)
            pass
        if time.time() - start_time > timeout_finish:
            raise TimeoutError("Lean4 REPL timed out.")
        # time.sleep(0.1)

    # kill
    if kill:
        child.close()
    return json.loads(res)


def setup_repl():
    child = pexpect.spawn(
        f"{DEFAULT_LAKE_PATH} exe repl",
        cwd=DEFAULT_LEAN_WORKSPACE)

    # Use the unprotected version to avoid error-loops.
    res = _send_code_read_json(
        {
            "cmd": LEAN4_DEFAULT_HEADER,
            "allTactics": True,
            "tactics": True,
        },
        _child=child
    )
    print("setup res")
    print(res)
    return child

In [16]:
os.getcwd()

'/home/ubuntu/ohm-tree-filesys-tokyo/plasma-converter'

In [13]:
os.chdir('/home/ubuntu/ohm-tree-filesys-tokyo/plasma-converter')

In [24]:
setup_repl()

cmd_json {"cmd": "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n", "allTactics": true, "tactics": true}


In [51]:
code = '''/-- If $f(x)=5x-12$, find a value for $x$ so that $f^{-1}(x)=f(x+1)$. Show that it is \frac{47}{24}.-/
theorem mathd_algebra_422 (x : ℝ) (σ : Equiv ℝ ℝ) (h₀ : ∀ x, σ.1 x = 5 * x - 12)
  (h₁ : σ.1 (x + 1) = σ.2 x) : x = 47 / 24 := by
  -- Simplify the given conditions using the properties of the function and its inverse.
  simp_all [Equiv.eq_symm_apply, mul_add, mul_one, sub_eq_iff_eq_add, add_comm]
  -- Apply the definition of the inverse function to get an equation involving σ.1 and σ.2.
  apply_fun σ.1 at h₁
  -- Normalize the equation to simplify it further.
  ring_nf at h₁ ⊢
  -- Simplify the equation using the definition of σ.1.
  field_simp [h₀] at h₁ ⊢
  -- Simplify the numerical constants in the equation.
  -- Solve the linear equation to find the value of x.
  linarith
'''

In [52]:
print(code)

/-- If $f(x)=5x-12$, find a value for $x$ so that $f^{-1}(x)=f(x+1)$. Show that it is rac{47}{24}.-/
theorem mathd_algebra_422 (x : ℝ) (σ : Equiv ℝ ℝ) (h₀ : ∀ x, σ.1 x = 5 * x - 12)
  (h₁ : σ.1 (x + 1) = σ.2 x) : x = 47 / 24 := by
  -- Simplify the given conditions using the properties of the function and its inverse.
  simp_all [Equiv.eq_symm_apply, mul_add, mul_one, sub_eq_iff_eq_add, add_comm]
  -- Apply the definition of the inverse function to get an equation involving σ.1 and σ.2.
  apply_fun σ.1 at h₁
  -- Normalize the equation to simplify it further.
  ring_nf at h₁ ⊢
  -- Simplify the equation using the definition of σ.1.
  field_simp [h₀] at h₁ ⊢
  -- Simplify the numerical constants in the equation.
  -- Solve the linear equation to find the value of x.
  linarith



In [42]:
code2 = '''
/-- Find the greatest common factor of 180 and 168. Show that it is 12.-/
theorem mathd_numbertheory_188 : Nat.gcd 180 168 = 12 := by
  -- Use the recursive definition of the greatest common divisor to simplify the problem.
  rw [Nat.gcd_rec]
'''

In [78]:
example = "example : f = 2 := rfl"
example_def = "def f : Nat := by have t := 37; exact t"

In [81]:
child = setup_repl()
result = send_code_read_json({
    "cmd": example_def,
    "allTactics": True,
    "tactics": True,
    "env": 0
}, _child=child)
print(result)

cmd_json {"cmd": "", "allTactics": true, "tactics": true}
setup res
{'env': 0, 'ast': []}
compiling
def f : Nat := by have t := 37; exact t
cmd_json {"cmd": "def f : Nat := by have t := 37; exact t", "allTactics": true, "tactics": true, "env": 0}
{'tactics': [{'tactic': 'have t := 37', 'proofState': 0, 'pos': {'line': 1, 'column': 18}, 'goals': '⊢ Nat', 'endPos': {'line': 1, 'column': 30}}, {'tactic': 'exact t', 'proofState': 1, 'pos': {'line': 1, 'column': 32}, 'goals': 't : Nat ⊢ Nat', 'endPos': {'line': 1, 'column': 39}}], 'env': 1, 'ast': []}


Try more basic code

child = setup_repl()
result = send_code_read_json({
    "cmd": input_data['task'],
    "allTactics": True,
    "tactics": True,
    "env": 0
}, _child=child)